# SHA CBC

We are given a huge encoded file and a python script that tells us how it was encrypted. As the saying goes, "It's like AES-CBC but with SHAKE-128 instead." In any case, it's basically 100000 copies of the flag, all encrypted.

In [1]:
allbytes = open('flag.enc', 'rb').read()
len(allbytes)

6300000

Great, our `naked_flag` must have length 63. The CBC part of this encryption means that `current_enc_block = hash(prev_enc_block ^ current_raw_block)`. Since it is infeasible to reverse the hash, the challenge here must be to find two identical values of `current_enc_block` that have a different `prev_enc_block`, so that we know the XOR of the two raw blocks. This is made particularly difficult by the fact that the hash might not be injective. Specifically, there are two types of collisions we might find:
1. An actual hash collision $hash(i)=hash(j)$ with $i\neq j$.
2. Two hashes of the same XORed block, but with a different `prev_enc_block`, and this is what gives us information.

Still, the first step is to find all `current_enc_block` values that take multiple distinct (`position % 63`, `prev_enc_block`) tuples.

In [2]:
from Crypto.Util.strxor import strxor
from itertools import combinations

dic = {}
for i in range(4, len(allbytes), 4):
    dic.setdefault(allbytes[i:i+4], set()).add((i % 63, allbytes[i-4:i]))
good = [(a,c,strxor(b,d)) for v in dic.values() for (a,b),(c,d) in combinations(v,2)]
print(f'{len(good)=}')
[(a,b,c.hex()) for a,b,c in good][:10]

len(good)=130


[(14, 59, '870f0dfe'),
 (61, 48, '331c6f07'),
 (0, 15, '8e980296'),
 (14, 12, 'b832cb8c'),
 (26, 22, '10186508'),
 (25, 39, '1d05a005'),
 (57, 45, '736de627'),
 (39, 16, '31480802'),
 (46, 44, '9d203e49'),
 (22, 58, '13758ae6')]

We find that there are 130 such collisions, though we don't know which type is which. In that case, the best chance of getting the right flag is to just maximise number of blocks that satisfy the second condition.

We will use z3 to maximise the number of satisfied predicates.

In [3]:
from z3 import *
s = Optimize()
bv = [BitVec(f'b{i}', 8) for i in range(63)]
s.add(And([Or([b == c for c in b'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ']) for b in bv]))
total = sum(If(And([bv[(a+i)%63]^bv[(b+i)%63]==z[i] for i in range(4)]),1,0) for a,b,z in good)
s.maximize(total)
assert s.check() == sat
f"CTFSG{{{''.join(chr(s.model()[b].as_long()) for b in bv)}}}"

'CTFSG{Oh I gUesS I hAVe alwAys likEd pOurIng ThingS inTo oTher ThiNgs}'

And there's our flag! Just for fun, let's see how many of our 130 collisions are of the second type:

In [4]:
s.model().evaluate(total)

63